## Required Libraries

In [ ]:
! pip install git+https://github.com/rcmalli/keras-vggface.git
! pip install keras_applications --no-deps
filename = "/usr/local/lib/python3.7/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))
import tensorflow as tf
from keras_vggface.vggface import VGGFace
from keras_vggface import utils

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-bw8fujzb
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-bw8fujzb


In [ ]:
%tensorflow_version 2.x
import tensorflow
tensorflow.__version__

from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization
from tensorflow.keras.layers import UpSampling2D, Input, Concatenate
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
import numpy as np
from google.colab import files
import os
import cv2 as cv
from matplotlib import image
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# Classification instruments
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier 
from sklearn.utils import shuffle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path="drive/MyDrive/Progetto DIGITAL/02_Classification - 2D/Faces/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Classification

In [ ]:
def identity(input):
    return input

# Data loader for binary classification
def load_data(feature_extractor=identity):
   
    images = []
   
    labels_A = []
    labels_B = []
    labels_G = []

    for f in sorted(os.listdir("/content/"+ path)):
        if f.endswith('.jpg'):
            img = cv.imread("/content/"+ path + f)
            
            # The VGGFace algorithm requires the faces to be resized to 224 x 224 pixels
            img = cv.resize(img,(224,224)) 

            if feature_extractor != identity:
                img=img_to_array(img)
                img=np.expand_dims(img,axis=0)
                img=utils.preprocess_input(img)

                img_encode = feature_extractor(img)
                images.append(np.squeeze(K.eval(img_encode)).tolist())
            else:
                images.append(img)
            
            # Classes
            label = f.split('_')[0]
            if label == 'Anna':
               labels_A.append(1)
               labels_B.append(0)
               labels_G.append(0)
            elif label == 'Beatrice':
               labels_A.append(0)
               labels_B.append(1)
               labels_G.append(0)
            elif label == 'Guglielmo':
               labels_A.append(0)
               labels_B.append(0)
               labels_G.append(1)
            else:
               labels_A.append(0)
               labels_B.append(0)
               labels_G.append(0)

    # Converting to array
    images = np.asarray(images)
    labels_A = np.asarray(labels_A)
    labels_B = np.asarray(labels_B)
    labels_G = np.asarray(labels_G)

    # Converting the data type
    images = images.astype('float32')/255.0

    # Train e test sets
    X_train, X_test, y_train_A, y_test_A = train_test_split(images, labels_A, test_size=0.1, random_state=1)
    X_train, X_test, y_train_B, y_test_B = train_test_split(images, labels_B, test_size=0.1, random_state=1)
    X_train, X_test, y_train_G, y_test_G = train_test_split(images, labels_G, test_size=0.1, random_state=1)

    return np.array(X_train), np.array(X_test), np.array(y_train_A), np.array(y_train_B), np.array(y_train_G), np.array(y_test_A), np.array(y_test_B), np.array(y_test_G)
    

In [ ]:
X_train, X_test, y_train_A, y_train_B, y_train_G, y_test_A, y_test_B, y_test_G = load_data()

In [ ]:
print("Training samples: "+str(len(X_train)))
print("Validation samples: "+str(len(X_test)))

Training samples: 396
Validation samples: 45


In [ ]:
# For NNet models
pix = np.shape(X_train)[1]

In [ ]:
le = LabelEncoder()
y_train_A = tf.keras.utils.to_categorical(le.fit_transform(y_train_A))
y_train_B = tf.keras.utils.to_categorical(le.fit_transform(y_train_B))
y_train_G = tf.keras.utils.to_categorical(le.fit_transform(y_train_G))
y_test_A = tf.keras.utils.to_categorical(le.fit_transform(y_test_A))
y_test_B = tf.keras.utils.to_categorical(le.fit_transform(y_test_B))
y_test_G = tf.keras.utils.to_categorical(le.fit_transform(y_test_G))

In [ ]:
# Models

def data_augmentation(hidden_dim=512):
    
    # Explicit input layer
    inputs = keras.Input(shape=(224, 224, 3))
    x = inputs

    x = keras.layers.RandomFlip(mode='horizontal_and_vertical')(x)  
    x = keras.layers.RandomRotation(factor=0.3)(x) 
    outputs = keras.layers.RandomContrast(factor=0.5)(x) 

    model = Model(inputs=inputs,outputs=outputs)

    return model

def vgg_finetuning(hidden_dim=512):
   
    vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
    inputs = vgg_model.input

    # freeze 
    for layer in vgg_model.layers:
        layer.trainable = False

    last_layer = vgg_model.get_layer('pool5').output
    x = Flatten(name='flatten')(last_layer)
    x = Dense(hidden_dim, activation='relu', name='fc6')(x)
    x = Dense(hidden_dim, activation='relu', name='fc7')(x)
    output_Anna= Dense(2, activation='softmax', name='Anna')(x)
    output_Beatrice= Dense(2, activation='softmax', name='Beatrice')(x)
    output_Gugliemo = Dense(2, activation='softmax', name='Gugliemo')(x)
    outputs = [output_Anna,output_Beatrice,output_Gugliemo]

    model = Model(inputs=inputs,outputs=outputs)

    return model

def vgg_features(hidden_dim=512):
   
    vgg_model = VGGFace(include_top=True, input_shape=(224, 224, 3))
    inputs = vgg_model.input

    # freeze 
    for layer in vgg_model.layers:
        layer.trainable = False

    x = vgg_model.get_layer('fc7/relu').output
    output_Anna= Dense(2, activation='softmax', name='Anna')(x)
    output_Beatrice= Dense(2, activation='softmax', name='Beatrice')(x)
    output_Gugliemo = Dense(2, activation='softmax', name='Gugliemo')(x)
    outputs = [output_Anna,output_Beatrice,output_Gugliemo]

    model = Model(inputs=inputs,outputs=outputs)

    return model

In [ ]:
prova = data_augmentation()
prova.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 random_flip (RandomFlip)    (None, 224, 224, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 224, 224, 3)      0         
 tion)                                                           
                                                                 
 random_contrast (RandomCont  (None, 224, 224, 3)      0         
 rast)                                                           
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def model_training(build_model,data,augmentation=False):

    if augmentation:
        DA = data_augmentation()
        # Applying data augmentation to training set
        data = DA.predict(data)

    model = build_model()

    losses = {"Anna": "binary_crossentropy",
            "Beatrice": "binary_crossentropy",
            "Gugliemo": "binary_crossentropy"}

    model.compile(loss=losses,
                optimizer='Adam',
                metrics=['accuracy'])
    
    history = model.fit(data,[y_train_A, y_train_B, y_train_G], 
                    validation_data=(X_test, [y_test_A, y_test_B, y_test_G]), 
                    batch_size=10, epochs=20, verbose=1)
    
    return model, history

In [ ]:
classifier, history = model_training(vgg_features,X_train)

Epoch 1/20
40/40 [==============================] - 14s 146ms/step - loss: 1.8801 - Anna_loss: 0.6319 - Beatrice_loss: 0.6352 - Gugliemo_loss: 0.6130 - Anna_accuracy: 0.9722 - Beatrice_accuracy: 0.8788 - Gugliemo_accuracy: 0.9798 - val_loss: 1.6994 - val_Anna_loss: 0.5664 - val_Beatrice_loss: 0.5947 - val_Gugliemo_loss: 0.5383 - val_Anna_accuracy: 1.0000 - val_Beatrice_accuracy: 0.8667 - val_Gugliemo_accuracy: 1.0000
Epoch 2/20
40/40 [==============================] - 2s 60ms/step - loss: 1.5493 - Anna_loss: 0.5314 - Beatrice_loss: 0.5316 - Gugliemo_loss: 0.4862 - Anna_accuracy: 0.9874 - Beatrice_accuracy: 0.9596 - Gugliemo_accuracy: 0.9949 - val_loss: 1.4207 - val_Anna_loss: 0.4741 - val_Beatrice_loss: 0.5151 - val_Gugliemo_loss: 0.4315 - val_Anna_accuracy: 1.0000 - val_Beatrice_accuracy: 0.9111 - val_Gugliemo_accuracy: 1.0000
Epoch 3/20
40/40 [==============================] - 2s 61ms/step - loss: 1.2984 - Anna_loss: 0.4524 - Beatrice_loss: 0.4525 - Gugliemo_loss: 0.3935 - Anna_accur

In [ ]:
y_test_list = [y_test_A, y_test_B, y_test_G]
names = ['Anna', 'Beatrice', 'Guglielmo']
pos = 0

y_pred = classifier.predict(X_test)
for y in y_test_list:
  print("\n -", names[pos])
  y_pred_ = np.argmax(y_pred[pos], axis = 1)
  y_test = np.argmax(y_test_list[pos], axis = 1)
  print(classification_report(y_test, y_pred_, digits = 2))
  pos+=1


 - Anna
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        34
           1       1.00      1.00      1.00        11

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45


 - Beatrice
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00        18

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45


 - Guglielmo
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       1.00      1.00      1.00        14

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00     

In [ ]:
# Save model
classifier.save('NN_BinaryClassifier_2D.h5')

!cp NN_BinaryClassifier_2D.h5 '/content/drive/MyDrive/Progetto DIGITAL/02_Classification - 2D/Modelli/'
#files.download('NN_BinaryClassifier_2D.h5')

### With data augmentation

In [ ]:
classifier, history = model_training(vgg_features,X_train,augmentation=True)

Epoch 1/20
40/40 [==============================] - 5s 72ms/step - loss: 1.8871 - Anna_loss: 0.6386 - Beatrice_loss: 0.6324 - Gugliemo_loss: 0.6161 - Anna_accuracy: 0.9470 - Beatrice_accuracy: 0.9520 - Gugliemo_accuracy: 0.9747 - val_loss: 1.6948 - val_Anna_loss: 0.5684 - val_Beatrice_loss: 0.5881 - val_Gugliemo_loss: 0.5383 - val_Anna_accuracy: 1.0000 - val_Beatrice_accuracy: 0.9111 - val_Gugliemo_accuracy: 1.0000
Epoch 2/20
40/40 [==============================] - 2s 62ms/step - loss: 1.5474 - Anna_loss: 0.5337 - Beatrice_loss: 0.5280 - Gugliemo_loss: 0.4857 - Anna_accuracy: 0.9848 - Beatrice_accuracy: 0.9495 - Gugliemo_accuracy: 0.9949 - val_loss: 1.4199 - val_Anna_loss: 0.4772 - val_Beatrice_loss: 0.5117 - val_Gugliemo_loss: 0.4310 - val_Anna_accuracy: 1.0000 - val_Beatrice_accuracy: 0.9111 - val_Gugliemo_accuracy: 1.0000
Epoch 3/20
40/40 [==============================] - 2s 62ms/step - loss: 1.2943 - Anna_loss: 0.4529 - Beatrice_loss: 0.4486 - Gugliemo_loss: 0.3928 - Anna_accurac

In [ ]:
y_test_list = [y_test_A, y_test_B, y_test_G]
names = ['Anna', 'Beatrice', 'Guglielmo']
pos = 0

y_pred = classifier.predict(X_test)
for y in y_test_list:
  print("\n -", names[pos])
  y_pred_ = np.argmax(y_pred[pos], axis = 1)
  y_test = np.argmax(y_test_list[pos], axis = 1)
  print(classification_report(y_test, y_pred_, digits = 2))
  pos+=1


 - Anna
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        34
           1       1.00      1.00      1.00        11

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45


 - Beatrice
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00        18

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45


 - Guglielmo
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       1.00      1.00      1.00        14

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00     

In [ ]:
# Save model
classifier.save('NN_BinaryClassifier_DA_2D.h5')

!cp NN_BinaryClassifier_DA_2D.h5 '/content/drive/MyDrive/Progetto DIGITAL/02_Classification - 2D/Modelli/'
#files.download('NN_BinaryClassifier_2D.h5')

# Altri modelli testati

In [ ]:
# Data loader
def load_data():
    labels = []
    images = []

    for f in sorted(os.listdir("/content/"+ path)):
        if f.endswith('.jpg'):
            img = cv.imread("/content/"+ path + f)
            
            # The VGGFace algorithm requires the faces to be resized to 224 x 224 pixels
            img = cv.resize(img,(224,224)) 
            images.append(img)
            
            # Classes
            label = f.split('_')[0]
            labels.append(label)

    # Converting to array
    images = np.asarray(images)
    labels = np.asarray(labels)

    # Converting the data type
    images = images.astype('float32')/255.0

    # Train e test sets
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1)

    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

In [ ]:
X_train, X_test, y_train, y_test = load_data()

In [ ]:
print("Training samples: "+str(len(X_train)))
print("Validation samples: "+str(len(X_test)))

In [ ]:
# For NNet models
pix = np.shape(X_train)[1]
classes = len(np.unique(y_train))

In [ ]:
# one hot encoding del target
le = LabelEncoder()
y_train_transformed = tf.keras.utils.to_categorical(le.fit_transform(y_train))
y_test_transformed = tf.keras.utils.to_categorical(le.fit_transform(y_test))

### VGGFace Finetuning with data augmentation layers

In [ ]:
def data_augmentation_model(hidden_dim=512):
    
    # Explicit input layer
    inputs = keras.Input(shape=(224, 224, 3))
    x = inputs

    x = keras.layers.RandomFlip(mode='horizontal_and_vertical')(x)  
    x = keras.layers.RandomRotation(factor=0.3)(x) 
    outputs = keras.layers.RandomContrast(factor=0.5)(x) 

    return inputs, outputs

def vgg_pre_trained(hidden_dim=512):
   
    vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
    inputs = vgg_model.input

    # freeze 
    for layer in vgg_model.layers:
        layer.trainable = False

    last_layer = vgg_model.get_layer('pool5').output
    x = Flatten(name='flatten')(last_layer)
    x = Dense(hidden_dim, activation='relu', name='fc6')(x)
    x = Dense(hidden_dim, activation='relu', name='fc7')(x)
    outputs = Dense(classes, activation='softmax', name='fc8')(x)

    return inputs, outputs


In [ ]:
# data_augmentation_model
inputs, outputs = data_augmentation_model()
data_augmentation = Model(inputs=inputs,outputs=outputs)
# vgg_pre_trained
inputs, outputs = vgg_pre_trained()
vgg_model=Model(inputs=inputs,outputs=outputs)

In [ ]:
data_augmentation.summary()

In [ ]:
vgg_model.summary()

In [ ]:
model = Sequential(
    [ data_augmentation,
      vgg_model
     ]
)

model.compile(loss=keras.losses.categorical_crossentropy,
            optimizer='Adam',
            metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train_transformed,
            epochs=10, batch_size=10, 
            validation_data=(X_test, y_test_transformed))

In [ ]:
predictions = model.predict(X_test)

In [ ]:
cm = confusion_matrix(np.argmax(y_test_transformed, axis=1), np.argmax(predictions,axis=1))
print(cm)
plt.imshow(cm, cmap=plt.cm.Blues)